In [58]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

Fase 1: Exploración y Limpieza
1.
Exploración Inicial:
Realiza una exploración inicial de los datos para identificarposibles problemas, como valores nulos, atípicos o datosfaltantes en las columnas relevantes.
Utiliza funciones de Pandas para obtener informaciónsobre la estructura de los datos, la presencia de valoresnulos y estadísticas básicas de las columnas involucradas.
Une los dos conjuntos de datos de la forma más eficiente.

In [59]:
df_CLH = pd.read_csv("Customer Loyalty History.csv", index_col=None)
df_CFA = pd.read_csv("Customer Flight Activity.csv", index_col=None)

In [60]:
# Función para exploración general de datos
def exploracion_datos(df):
    print('_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________\n')
    print(df.info())
    
    print('___________________ FORMA DEL DATAFRAME ____________________\n')
    print(f"El número de filas que tenemos es de {df.shape[0]}.\nEl número de columnas es de {df.shape[1]}\n")
    
    print('_______________ NULOS, ÚNICOS Y DUPLICADOS _________________\n')
    print('La cantidad de valores NULOS por columna es de:\n')
    print(df.isnull().sum())
    print('____________________________________________________________\n')

    print('El porcentaje de valores NULOS por columna es de:\n')
    porcentaje_nulos = (df.isnull().sum() / df.shape[0]) * 100
    porcentaje_nulos = porcentaje_nulos.round(2)
    print(porcentaje_nulos)
    print('____________________________________________________________\n')

    print('La cantidad de valores ÚNICOS por columna es de:\n')
    for columna in df.columns:
        cantidad_unicos = df[columna].nunique()
        print(f'La columna {columna} tiene {cantidad_unicos} valores únicos.')
    print('____________________________________________________________\n')

    print('La cantidad de valores DUPLICADOS por columna es de:\n')
    for columna in df.columns:
        cantidad_duplicados = df.duplicated(subset=columna).sum()
        print(f'La columna {columna} tiene {cantidad_duplicados} valores duplicados.')
    print('____________________________________________________________\n')
    
    print('____________________ RESUMEN ESTADÍSTICO ____________________\n')
    print('___________________ Variables Numéricas _____________________\n')
    if df.select_dtypes(include=[float, int]).empty:
        print("No hay variables numéricas para mostrar el resumen estadístico.")
    else:
        print(df.describe().T)
    print('____________________________________________________________\n')
    print('___________________ Variables Categóricas _____________________\n')
    if df.select_dtypes(include=[object]).empty:
        print("No hay variables categóricas para mostrar el resumen estadístico.")
    else:
        print('__________________ Variables Categóricas ____________________\n')
        print(df.describe(include='object').T)

In [61]:
# Llamamos a la función para explorar el dataframe df_CLH 
exploracion_datos(df_CLH)

_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   floa

In [62]:
# Vemos que aspecto tienen las 5 primeras columnas del dataframe df_CLH 
df_CLH.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [63]:
# Llamamos a la función para explorar el dataframe df_CFA
exploracion_datos(df_CFA)

_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB
None
___________________ FORMA DEL DATAFRAME ____________________

E

In [64]:
df_CFA.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [68]:
print(df_CFA[Loyalty Number])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1265992228.py, line 1)